Database operations to build the structure for visualization

I will need 4 groups of data:
* topics (aggregator)
* docs
* words
* people

In [9]:
import pandas as pd
import json
import os
import pickle
import re
import math
import gensim
from collections import Counter

In [2]:
outputs = '../outputs/'

In [3]:
file_tables = '../outputs/tables_dict.pkl'
cpdoc_tables = pickle.load(open(file_tables, 'rb'))

In [4]:
list(cpdoc_tables)

['person_doc ',
 'top_persons',
 'persons',
 'top_countries',
 'docs',
 'top_topics',
 'doc_counts ',
 'countries',
 'top_classifications',
 'topic_doc',
 'topics',
 'country_doc']

In [41]:
docs = cpdoc_tables['docs']

docs['length'] = docs['body'].apply(lambda x: len(x.split()))

In [42]:
docs.head()

,id,volume_id,main_language,readability,title,keywords,date,collection,pdf,body,length
0,ag_1973.11.20_doc_I-1,ag_1973.11.20,none,0.40,Documentos felicitando Azeredo da Silveira pel...,Felicitações e congratulações; Ministério das ...,1973-11-15,CPDOC,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3...",261
1,ag_1973.11.20_doc_I-100,ag_1973.11.20,pt,NaN,Documentos felicitando Azeredo da Silveira pel...,Felicitações e congratulações; Ministério das ...,1974-02-22,CPDOC,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"mjm h\n\n. . . ' "" . ww and\n& !\nzczc tea363 ...",173
2,ag_1973.11.20_doc_I-101,ag_1973.11.20,pt,NaN,Documentos felicitando Azeredo da Silveira pel...,Felicitações e congratulações; Ministério das ...,1974-02-22,CPDOC,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n \n \n\n \n\n \n\n114"":\n\n \n\n ...",72
3,ag_1973.11.20_doc_I-102,ag_1973.11.20,pt,NaN,Documentos felicitando Azeredo da Silveira pel...,Felicitações e congratulações; Ministério das ...,1974-02-22,CPDOC,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n"" "", ""m-nmfvww mmm/""ar? &"", wa "" ""***-**""...",131
4,ag_1973.11.20_doc_I-103,ag_1973.11.20,pt,0.59,Documentos felicitando Azeredo da Silveira pel...,Felicitações e congratulações; Ministério das ...,1974-02-22,CPDOC,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nmi*/gm\n%adazw/sõ'\n\n%% && %%%&\n\n...,295


In [43]:
set(list(cpdoc_tables['docs']['main_language']))

{'de', 'en', 'es', 'fr', 'none', 'pt'}

In [44]:
data = []
for index, row in cpdoc_tables['docs'].fillna(-1).iterrows():
    temp_dict = {'id': row['id'], 'main language': row['main_language'], 'readability': row['readability'], \
                 'date': str(row['date']), 'length': row['length']}
    data.append(temp_dict)

In [45]:
file_path = outputs+'docs.json'
json.dump(data, open(file_path, 'w'))

# docs and topics

In [204]:
topic_doc = cpdoc_tables['topic_doc']
topic_doc.head()

,doc_id,topic_id,topic_score,date
0,ag_1973.11.20_doc_I-100,5009,0.167579,1974-02-22
1,ag_1973.11.20_doc_I-100,5010,0.040318,1974-02-22
2,ag_1973.11.20_doc_I-100,5040,0.092381,1974-02-22
3,ag_1973.11.20_doc_I-100,5041,0.090368,1974-02-22
4,ag_1973.11.20_doc_I-100,5051,0.029285,1974-02-22


# docs and persons

In [205]:
persons = cpdoc_tables['persons']
persons = persons.rename(index=str, columns={"id": "person_id"})

In [206]:
person_doc = cpdoc_tables['person_doc ']
person_doc = pd.merge(person_doc, persons, on='person_id', how='inner')
person_doc = person_doc.drop(columns=['date', 'source', 'birth_year', 'death_year', 'description'])
person_doc = person_doc.replace('(.*),(.*),.*', r'\1,\2', regex=True)
person_doc = person_doc.replace('(.*), (.*)', r'\2 \1', regex=True)

In [207]:
#remove special characters
person_doc = person_doc.name.apply(pd.Series) \
    .replace('[“”]', '', regex=True) \
    .replace('[áàãâ]', 'a', regex=True) \
    .replace('[óòõô]', 'o', regex=True) \
    .replace('[éèê]', 'e', regex=True) \
    .replace('[íì]', 'i', regex=True) \
    .replace('[úù]', 'u', regex=True) \
    .replace('ç', 'c', regex=True) \
    .merge(person_doc, left_index = True, right_index = True) \
    .drop("name", axis=1) \
    .rename(columns={0: "name"})

In [208]:
person_doc.head()

,name,person_id,doc_id,person_count
0,Dean Gooderham Acheson,100070,ag_1977.07.00_doc_I-4,1
1,John Quincy Adams,100097,be_1974.03.15_doc_XI-2,1
2,Konrad Adenauer,100118,be_1974.03.15_doc_X-21,1
3,Konrad Adenauer,100118,be_1977.06.01_doc_II-28,1
4,Konrad Adenauer,100118,pn_1974.08.15_doc_I-3,1


In [354]:
person_doc_count = pd.DataFrame(person_doc.groupby(['name'])['person_count'].agg('count')) #.apply(sum)
person_doc_count = person_doc_count.reset_index()

# docs and tokens

In [209]:
docs = cpdoc_tables['docs'][['id', 'title', 'pdf', 'body']]
docs = docs.rename(index=str, columns={"id": "doc_id"})
docs['length'] = docs['body'].apply(lambda x: len(x.split()))
docs.head()

,doc_id,title,pdf,body,length
0,ag_1973.11.20_doc_I-1,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\njr, /h ' ""& m$ hb'h'qo\n& la'?\n\n, é %%3...",261
1,ag_1973.11.20_doc_I-100,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"mjm h\n\n. . . ' "" . ww and\n& !\nzczc tea363 ...",173
2,ag_1973.11.20_doc_I-101,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n \n \n\n \n\n \n\n114"":\n\n \n\n ...",72
3,ag_1973.11.20_doc_I-102,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,"\n\n"" "", ""m-nmfvww mmm/""ar? &"", wa "" ""***-**""...",131
4,ag_1973.11.20_doc_I-103,Documentos felicitando Azeredo da Silveira pel...,http://www.fgv.br/cpdoc/acervo/arquivo-pessoal...,\n\n \n\nmi*/gm\n%adazw/sõ'\n\n%% && %%%&\n\n...,295


# words and topics

In [210]:
computer = input("home OR fgv OR broken? ") 
if computer == 'home': outer_inputs = 'D:/'
elif computer == 'broken': outer_inputs = 'C:/Users/marcelo.old-note/Documents/'
else: print('computer is not defined')

home OR fgv OR broken? 
computer is not defined


In [211]:
file_lda_model = outer_inputs+'azeredo_files/model_lda_100_rs_00.pkl'
lda_model = pickle.load(open(file_lda_model, 'rb'))

In [212]:
def remove_special_char(text):
    text = re.sub('[áàãâ]', 'a', text)
    text = re.sub('[óòõô]', 'o', text)
    text = re.sub('[éèê]', 'e', text)
    text = re.sub('[íì]', 'i', text)
    text = re.sub('[úù]', 'u', text)
    text = re.sub('ç', 'c', text)
    return text

In [ ]:
#old version without sorting and selecting only 5 main tokens
def retrieve_tokens(text, token_score):
    main_tokens = []
    for token in token_score['tokens']:
        if token in text:
            token = remove_special_char(token) # refatorar (acaba repetindo mesma limpeza muitas vezes). ou talvez seja melhor assim para nao limpar texto inteiro desnecessariamente...
            main_tokens.append(token)
    return main_tokens